<a href="https://colab.research.google.com/github/acedesci/CNN_sentence/blob/master/S8_9_retail_analytics/S8_Module1_Retail_Demand_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas
import sklearn
import numpy
from sklearn import *

Import data

- here we can try to split data into different product groups -

In [35]:
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/S8_9_retail_analytics/salesCereals.csv'

salesCereals = pandas.read_csv(url)
salesCereals['PRICE_p2'] = salesCereals['PRICE']**2
salesCereals.head()

,Unnamed: 0,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY,Desc,Category,Sub-Category,SUMPRICE,COUNTPRICE,AVGPRICE,RELPRICE,PRICE_p2
0,6,2009-01-14,367.0,1111085319,14.0,13.0,13.0,26.32,1.88,1.88,0.0,0.0,0.0,PL HONEY NUT TOASTD OATS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,0.673490,3.5344
1,8,2009-01-14,367.0,1111085350,35.0,27.0,25.0,69.30,1.98,1.98,0.0,0.0,0.0,PL BT SZ FRSTD SHRD WHT,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,0.709314,3.9204
2,12,2009-01-14,367.0,1600027527,12.0,10.0,10.0,38.28,3.19,3.19,0.0,0.0,0.0,GM HONEY NUT CHEERIOS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,1.142784,10.1761
3,13,2009-01-14,367.0,1600027528,31.0,26.0,19.0,142.29,4.59,4.59,0.0,0.0,0.0,GM CHEERIOS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,1.644319,21.0681
4,14,2009-01-14,367.0,1600027564,56.0,48.0,42.0,152.32,2.72,3.07,1.0,0.0,0.0,GM CHEERIOS,COLD CEREAL,ALL FAMILY CEREAL,19.54,7,2.791429,0.974411,7.3984


In [36]:
print(salesCereals.groupby('UPC').count())
print(len(salesCereals['WEEK_END_DATE'].unique()))

            Unnamed: 0  WEEK_END_DATE  STORE_NUM  ...  AVGPRICE  RELPRICE  PRICE_p2
UPC                                               ...                              
1111085319         156            156        156  ...       156       156       156
1111085350         156            156        156  ...       156       156       156
1600027527         156            156        156  ...       156       156       156
1600027528         156            156        156  ...       156       156       156
1600027564         155            155        155  ...       155       155       155
3000006340         133            133        133  ...       133       133       133
3800031829         155            155        155  ...       155       155       155

[7 rows x 20 columns]
156


In [37]:
feature_list = ['PRICE', 'PRICE_p2', 'FEATURE', 'DISPLAY','TPR_ONLY','RELPRICE']

productList = salesCereals['UPC'].unique()
print(productList)

X, X_train, X_test = {}, {}, {}
y, y_train, y_test, y_pred = {}, {}, {}, {}

for upc in productList:
  
  X[upc] = salesCereals.loc[salesCereals['UPC']==upc][feature_list]
  y[upc] = salesCereals.loc[salesCereals['UPC']==upc]['UNITS']
  # Split into training and testing data
  X_train[upc], X_test[upc], y_train[upc], y_test[upc] = sklearn.model_selection.train_test_split(X[upc], y[upc], test_size=0.25, random_state=0)
  

[1111085319 1111085350 1600027527 1600027528 1600027564 3000006340
 3800031829]


In [38]:
#Linear model
regr = {}
regrSummary = pandas.DataFrame(columns=['MAE','MAPE', 'RMSE'], index = productList)

for upc in productList:
    regr[upc] = sklearn.linear_model.LinearRegression().fit(X_train[upc],y_train[upc])
    y_pred[upc] = regr[upc].predict(X_test[upc])
    
    testMAE = sklearn.metrics.mean_absolute_error(y_test[upc], y_pred[upc])
    testMAPE = numpy.mean(numpy.abs((y_test[upc] - y_pred[upc]) / y_test[upc]))
    testRMSE = numpy.sqrt(sklearn.metrics.mean_squared_error(y_test[upc], y_pred[upc]))
    regrSummary.loc[k] =  [testMAE, testMAPE, testRMSE]

print('Linear Regression Summary')
print(regrSummary)
print('average MAE:' + str(round(regrSummary['MAE'].mean(),2)))
print('average MAPE:' + str(round(regrSummary['MAPE'].mean(),2)))
print('average RMSE:' + str(round(regrSummary['RMSE'].mean(),2)))

Linear Regression Summary
                MAE      MAPE     RMSE
1111085319      NaN       NaN      NaN
1111085350      NaN       NaN      NaN
1600027527      NaN       NaN      NaN
1600027528      NaN       NaN      NaN
1600027564      NaN       NaN      NaN
3000006340      NaN       NaN      NaN
3800031829  6.57475  0.379121  8.51852
average MAE:6.57
average MAPE:0.38
average RMSE:8.52


In [41]:
#Tree model
regr = {}
regrSummary = pandas.DataFrame(columns=['MAE','MAPE', 'RMSE'], index = productList)

for upc in productList:
      
    regr[upc] = tree.DecisionTreeRegressor(random_state = 0).fit(X_train[upc],y_train[upc])
    y_pred[upc] = regr[upc].predict(X_test[upc])

    testMAE = sklearn.metrics.mean_absolute_error(y_test[upc], y_pred[upc])
    testMAPE = numpy.mean(numpy.abs((y_test[upc] - y_pred[upc]) / y_test[upc]))
    testRMSE = numpy.sqrt(sklearn.metrics.mean_squared_error(y_test[upc], y_pred[upc]))
    regrSummary.loc[upc] =  [testMAE, testMAPE, testRMSE]

print('Regression Tree Summary')
print(regrSummary)
print('average MAE:' + str(round(regrSummary['MAE'].mean(),2)))
print('average MAPE:' + str(round(regrSummary['MAPE'].mean(),2)))
print('average RMSE:' + str(round(regrSummary['RMSE'].mean(),2)))

Regression Tree Summary
                MAE      MAPE     RMSE
1111085319  7.10256  0.841491  10.7596
1111085350  5.92308  0.625815  7.47131
1600027527  15.5641  0.520292  30.3302
1600027528  8.64103  0.286716   13.221
1600027564  7.89744   0.38854  10.4538
3000006340      4.5  0.992923  7.76209
3800031829  7.48718  0.421318  9.13152
average MAE:8.16
average MAPE:0.58
average RMSE:12.73


In [42]:
import pickle

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
predDemand = pandas.read_csv(cwd + 'predictionInput_Prob1.csv')
print(predDemand)

    Unnamed: 0  avgPriceChoice         UPC  ...  DISPLAY  TPR_ONLY  RELPRICE
0            0             3.0  1600027528  ...        1         1  0.833333
1            1             3.0  1600027528  ...        1         1  1.000000
2            2             3.0  1600027528  ...        1         1  1.166667
3            3             3.0  1600027564  ...        1         1  0.833333
4            4             3.0  1600027564  ...        1         1  1.000000
5            5             3.0  1600027564  ...        1         1  1.166667
6            6             3.0  3000006340  ...        1         1  0.833333
7            7             3.0  3000006340  ...        1         1  1.000000
8            8             3.0  3000006340  ...        1         1  1.166667
9            9             3.0  3800031829  ...        1         1  0.833333
10          10             3.0  3800031829  ...        1         1  1.000000
11          11             3.0  3800031829  ...        1         1  1.166667

In [50]:
X = {}
y_pred = {}
predictedValueSeries = pandas.Series()
productList = predDemand['UPC'].unique()

for upc in productList:
  X[upc] = predDemand.loc[predDemand['UPC']==upc][feature_list] 
  # print(X[upc])
  upcIndex = predDemand.loc[predDemand['UPC']==upc].index
  y_pred[upc] = regr[upc].predict(X[upc]).clip(0)

  predictedValueSeries = predictedValueSeries.append(pandas.Series(y_pred[upc], index = upcIndex))

predDemand['predictSales'] = predictedValueSeries
print(predDemand.to_string())

    Unnamed: 0  avgPriceChoice         UPC  PRICE  PRICE_p2  FEATURE  DISPLAY  TPR_ONLY  RELPRICE  predictSales
0            0             3.0  1600027528    2.5      6.25        1        1         1  0.833333          76.0
1            1             3.0  1600027528    3.0      9.00        1        1         1  1.000000          57.0
2            2             3.0  1600027528    3.5     12.25        1        1         1  1.166667          59.0
3            3             3.0  1600027564    2.5      6.25        1        1         1  0.833333          70.0
4            4             3.0  1600027564    3.0      9.00        1        1         1  1.000000          58.0
5            5             3.0  1600027564    3.5     12.25        1        1         1  1.166667          58.0
6            6             3.0  3000006340    2.5      6.25        1        1         1  0.833333          24.0
7            7             3.0  3000006340    3.0      9.00        1        1         1  1.000000       

In [0]:
cwd = '/content/drive/My Drive/'
for k in productList:
    regr[k] = sklearn.linear_model.LinearRegression().fit(X[k],y[k])
    filename = cwd+str(k)+'_demand_model.sav'
    pickle.dump(regr[k], open(filename, 'wb'))

In [20]:
import pickle
regr = {}
for upc in productList:
    filename = cwd+str(upc)+'_demand_model.sav'
    print(filename)
    # load the model from disk
    regr[upc] = pickle.load(open(filename, 'rb'))

/content/drive/My Drive/1111085319_demand_model.sav
/content/drive/My Drive/1111085350_demand_model.sav
/content/drive/My Drive/1600027527_demand_model.sav
/content/drive/My Drive/1600027528_demand_model.sav
/content/drive/My Drive/1600027564_demand_model.sav
/content/drive/My Drive/3000006340_demand_model.sav
/content/drive/My Drive/3800031829_demand_model.sav
